In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

c:\Programming\python\kt_aivle\webtoon\myvenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
def find_sim(story1, story2, checkpoints):
    
    story_vector1 = checkpoints.encode(story1)
    story_vector2 = checkpoints.encode(story2)
    story_sims = util.cos_sim(story_vector1,story_vector2)
    return story_sims

In [7]:
import pymongo

conn = pymongo.MongoClient()
webtoon_db = conn.webtoon_db
webtoon_collection = webtoon_db.webtoon_collection

In [105]:

checkpoints = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
story_sim_list = []
cnt = 1
base = webtoon_collection.find({},{'title':1})
base_title_list = [b['title'] for b in base]
base = webtoon_collection.find({},{'story':1})
base_story_list = [b['story'] for b in base]
compare = webtoon_collection.find({},{'story':1,'title':1})
compare_title_list = [c['title'] for c in compare]
compare = webtoon_collection.find({},{'story':1,'title':1})
compare_story_list = [c['story'] for c in compare]
sim_list = find_sim(base_story_list,compare_story_list,checkpoints)
for idx,base_title in enumerate(base_title_list):
    sims = [(sim,t) for sim,t in zip(sim_list[idx],compare_title_list)]
    sims.sort(reverse=True)
    sim_dic = {title: story.item()for story, title in sims[1:11]}
    webtoon_collection.update_one({"title":sims[0][1]},{"$set":{"similarity":sim_dic}})
    